#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [6]:
import pymysql
from sqlalchemy import create_engine
driver = 'mysql+pymysql:'
ip = 'relational.fit.cvut.cz'
port = 3306
user = 'guest'
password = 'relational'
database = 'stats'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [7]:
engine = create_engine(connection_string)
query = 'SELECT * FROM users'

#### 4. Import the users table 

In [8]:
users_df = pd.read_sql(query, engine)

#### 5. Rename Id column to userId

In [15]:
users_df.rename(columns = {'Id': 'userId'}, inplace = True)
users_df.head(1)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None


#### 6. Import the posts table. 

In [16]:
query = 'SELECT * FROM posts'
posts_df = pd.read_sql(query, engine)

#### 7. Rename Id column to postId and OwnerUserId to userId

In [36]:
posts_df.rename(columns = {'Id': 'postId', 'OwnerUserId': 'userId'}, inplace = True)
posts_df.head(1)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [24]:
NewUsersdf = users_df[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
NewPostsdf = posts_df[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

In [42]:
NewUsersdf.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


In [41]:
NewPostsdf.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [39]:
MergedDB = NewUsersdf.merge(NewPostsdf, left_on = 'userId', right_on = 'userId')

#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [45]:
MergedDB.info()
#48396 missing values at ViewCount

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90584 entries, 0 to 90583
Data columns (total 9 columns):
userId          90584 non-null int64
Reputation      90584 non-null int64
Views           90584 non-null int64
UpVotes         90584 non-null int64
DownVotes       90584 non-null int64
postId          90584 non-null int64
Score           90584 non-null int64
ViewCount       42188 non-null float64
CommentCount    90584 non-null int64
dtypes: float64(1), int64(8)
memory usage: 6.9 MB


48396

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [49]:
#I Would fill the ViewCount with 0 because I think that NaN is like no visits to the post
MergedDB['ViewCount'].fillna(0, inplace = True)

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [50]:
MergedDB['ViewCount'].astype('int64')

0    0
1    0
2    0
3    0
4    0
Name: ViewCount, dtype: int64

In [51]:
MergedDB.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
